In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv('Expanded_Destinations.csv')  # Ensure path is correct for file access
expected_columns = ['Name', 'State', 'Type', 'BestTimeToVisit', 'Popularity']

# Check if all expected columns are present
if not all(col in df.columns for col in expected_columns):
    missing_columns = [col for col in expected_columns if col not in df.columns]
    raise KeyError(f"The following columns are missing from your dataset: {missing_columns}. "
                   f"Please ensure your dataset has these columns with the correct names.")

# Preprocess data using Label Encoding
label_encoders = {}
for column in ['Name', 'State', 'Type', 'BestTimeToVisit']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# If Popularity is categorical, encode it; if numeric, skip encoding
if df['Popularity'].dtype == 'object':
    popularity_encoder = LabelEncoder()
    df['Popularity'] = popularity_encoder.fit_transform(df['Popularity'])
else:
    popularity_encoder = None

# Split data into features and target
X = df[['State', 'Type', 'BestTimeToVisit', 'Popularity']]
y = df['Name']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the decision tree model
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Get user preferences
state_input = input("Enter the state to which you want to travel: ").title()
type_input = input("Enter the type of travel (e.g., Adventure, Culture): ").title()
time_input = input("Enter the best time you want to travel: ").title()
popularity_input = input("Enter the popularity level (e.g., Low, Medium, High): ").title()

# Validate and encode user input
user_input_data = {}
for column, value in zip(['State', 'Type', 'BestTimeToVisit'], [state_input, type_input, time_input]):
    if value not in label_encoders[column].classes_:
        print(f"Invalid input for {column}: {value}")
        print(f"Valid options for {column} are: {list(label_encoders[column].classes_)}")
        exit()  # Exit the program for invalid input
    user_input_data[column] = label_encoders[column].transform([value])[0]

# Handle popularity input
if popularity_encoder:  # If popularity is categorical
    if popularity_input not in popularity_encoder.classes_:
        print(f"Invalid input for Popularity: {popularity_input}")
        print(f"Valid options for Popularity are: {list(popularity_encoder.classes_)}")
        exit()
    user_input_data['Popularity'] = popularity_encoder.transform([popularity_input])[0]
else:  # If popularity is numeric
    try:
        user_input_data['Popularity'] = float(popularity_input)
    except ValueError:
        print(f"Invalid numeric value for Popularity: {popularity_input}")
        exit()

# Create DataFrame for user input
user_input = pd.DataFrame([user_input_data])

# Predict destination
predicted_index = model.predict(user_input)[0]
predicted_destination = label_encoders['Name'].inverse_transform([predicted_index])[0]

# Output the recommended destination
print("Recommended destination:", predicted_destination)


: 